In [28]:
!pip install transformers torch pandas

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [29]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Sat Jun 15 10:35:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0              28W /  70W |   2927MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [30]:
!ls /kaggle/input/


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


infloso-assignment  reviews


In [31]:
# Import necessary libraries
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline

In [32]:
# Define the dataset class
class ReviewsDataset(Dataset):
    def __init__(self, tokenizer, texts, max_len=512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = str(self.texts[index])
        text_enc = self.tokenizer.encode_plus(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )
        return {
            'input_ids': text_enc['input_ids'].flatten(),
            'attention_mask': text_enc['attention_mask'].flatten()
        }


In [33]:
# Load and preprocess data
def load_and_preprocess_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        reviews = file.read().strip().split('\n')
    preprocessed_reviews = [review.lower().strip() for review in reviews]
    return preprocessed_reviews

reviews = load_and_preprocess_data('/kaggle/input/reviews/reviews.txt')


In [34]:
# Setup tokenizer and data loader
tokenizer = T5Tokenizer.from_pretrained('t5-small')
dataset = ReviewsDataset(tokenizer, reviews)
data_loader = DataLoader(dataset, batch_size=2, shuffle=True)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [35]:
# Define and train model
def train_model(data_loader, model, device):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
    for epoch in range(3):  # Train for 3 epochs
        for batch in data_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            print(f"Epoch {epoch}, Loss: {loss.item()}")

model = T5ForConditionalGeneration.from_pretrained('t5-small')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
train_model(data_loader, model, device)


Epoch 0, Loss: 17.458782196044922
Epoch 0, Loss: 14.900627136230469
Epoch 0, Loss: 14.736344337463379
Epoch 0, Loss: 14.864032745361328
Epoch 0, Loss: 17.222349166870117
Epoch 0, Loss: 20.253862380981445
Epoch 0, Loss: 16.436100006103516
Epoch 0, Loss: 16.978046417236328
Epoch 0, Loss: 12.02389907836914
Epoch 0, Loss: 8.376055717468262
Epoch 0, Loss: 12.780811309814453
Epoch 0, Loss: 9.41207504272461
Epoch 0, Loss: 12.265164375305176
Epoch 0, Loss: 12.95509147644043
Epoch 0, Loss: 10.79813289642334
Epoch 0, Loss: 11.141063690185547
Epoch 0, Loss: 9.615585327148438
Epoch 0, Loss: 14.146730422973633
Epoch 0, Loss: 6.741063594818115
Epoch 0, Loss: 8.294747352600098
Epoch 0, Loss: 8.233511924743652
Epoch 0, Loss: 7.73572301864624
Epoch 0, Loss: 9.182097434997559
Epoch 0, Loss: 8.901589393615723
Epoch 0, Loss: 13.545244216918945
Epoch 0, Loss: 8.983579635620117
Epoch 0, Loss: 8.026556968688965
Epoch 0, Loss: 9.954524993896484
Epoch 0, Loss: 3.9236161708831787
Epoch 0, Loss: 7.64988803863525

In [39]:
# Summarize reviews
def summarize_reviews(reviews):
    summarizer = pipeline("summarization", model="t5-small")
    summaries = [summarizer(review, max_length=25, min_length=5, do_sample=False)[0]['summary_text'] for review in reviews[:5]]
    return summaries

summaries = summarize_reviews(reviews)
print("Generated Summaries:")
for summary in summaries:
    print(summary)


Generated Summaries:
infloso app gives a lot of opportunities for one to connect with brands and improve their social media presence 
it doesn't let me connect my instagram nor my youtube . nothing works.
i have an instagram family of close to 70k followers still your can't find me there? how
bahut hi vdiya app hai chote mote creator ko bi collaborate
i'm unable to add my youtube username.when ever i add it's telling wrong.


In [41]:
# Identify the top 5 longest reviews
top_reviews = sorted(reviews, key=len, reverse=True)[:5]
print("Top 5 Reviews Based on Length:")
for review in top_reviews:
    print(review)


Top 5 Reviews Based on Length:
poor user experience. i'm a software developer and being professional this app has the worst ux i've ever experienced. in profile section when i try to connect instagram, it get connected. but my username is same in all my social accounts and when i try to add another account (twitter,linkedin,youtube,etc..) with the same username then its reflecting back my previously added instagram account showing its already connected. seriously, this is the production level release you guys made for public?
i've been using infloso for a while now, and i can't express how much it has transformed my productivity. this app is a game-changer!firstly, the user interface is clean and intuitive. navigating through tasks, projects, and calendars is a breeze. the app's design encourages focus and efficiency.the task management features are top-notch. i love how i can easily create, prioritize, and track tasks across various projects. the ability to set deadlines and receive r

In [38]:
# Save the trained model
model.save_pretrained('/kaggle/working/summary_model')
print("Model saved successfully to /kaggle/working/summary_model")


Model saved successfully to /kaggle/working/summary_model
